In [ ]:
pip install gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
from zipfile import ZipFile
from bs4 import BeautifulSoup
import requests
import os 
from urllib.parse import urljoin
from PIL import Image

Building on the homework from session 6, I am gonna scrape images of the dragons instead of text about the dragons. 
Initially, I wanted to scrape from the /wiki/Dragon page, as that's were I scraped the urls from in the session 6 lab. However, when I tried to do this, the result was a lot of images not of dragons, but other motives like people and artifacts. 

I want as many of the images to be actual dragons, so I changed the main page to scrape urls from to be: /wiki/List_of_dragons_in_mythology_and_folklore. 

In [2]:
# putting the url in variable, 'url'
absolute_url = 'https://en.m.wikipedia.org'
relative_url = '/wiki/List_of_dragons_in_mythology_and_folklore'
#https://en.wikipedia.org/wiki/European_dragon
#https://en.wikipedia.org/wiki/Chinese_dragon
#https://en.wikipedia.org/wiki/List_of_tallest_trees

#absolute_url = 'https://commons.m.wikimedia.org'
#relative_url = '/w/index.php?go=Go&search=dragon&title=Special:MediaSearch&type=image'

url= absolute_url + relative_url          # scraping from the wikipedia dragons in mythology and folklore page 

# get the url 
r = requests.get(url)

# check the status
print("Status code: ", r.status_code)
#print(r.content)

Status code:  200


In [3]:
# create the soup! 
soup_urls = BeautifulSoup(r.content,'html.parser')
#soup.prettify()

In [4]:
# list to hold the scraped links
urls = []

# iterate through the main page and get href for each page and put in list
for url in soup_urls.find_all('a'):      # find all a tags containing hyper links
    href = url.get('href')               # get the href for each hyperlink on the website 
    url = absolute_url+href           
    urls.append(url)                     # append full link to the list of urls
print(len(urls), 'urls were scraped.')

643 urls were scraped.


In [5]:
# Create a folder to save the images
if not os.path.exists('images_dragons'):
    os.makedirs('images_dragons')

In [15]:
# find all the dragons! 
images_list = []

# find all image tags in each url 
for url in urls[100:300]:            # make list smaller 
  response = requests.get(url)       # make a request to get each scraped url
  soup_dragon = BeautifulSoup(response.content, 'html.parser')      # create new soup object to hold the images 
  images = soup_dragon.find_all('img',class_=["image_lazy_loaded",'thumbimage','image','sdms-image-result'])   # search for classes: image, thumbimage, sdms-image-result and image_lazy_loaded to get as many pictures as possible but not icons and other stuff 
  for image in images:               # iterate over found images 
    src = image.get('src')            
    alt = image.get('alt')         
    images_list.append({'src':src,'alt':alt})            # add the image information to the text
    if not src.startswith(('http://', 'https://')):      # check if the image url is a relative url 
      src = 'http://' + src.lstrip('/')
      src = urljoin(url, src)
      try: 
        img_res = requests.get(src)         # get the source 
        if img_res.status_code == 200:
        # Save images to folder  
          with open(f'images_dragons/{alt}.jpg', 'wb') as fout:
            fout.write(img_res.content)
        else:
          print(f"Error downloading image: {src}")
      except Exception as e:
        print(f"Error downloading image: {src} ({e})")
 
print(len(images_list),'dragons were found')

Error downloading image: http://upload.wikimedia.org/wikipedia/commons/thumb/e/e7/Utatlan2.jpg/220px-Utatlan2.jpg
Error downloading image: http://upload.wikimedia.org/wikipedia/commons/thumb/6/6a/Arnold_B%C3%B6cklin_-_Das_Irrlicht_-1882.jpeg/220px-Arnold_B%C3%B6cklin_-_Das_Irrlicht_-1882.jpeg
Error downloading image: http://upload.wikimedia.org/wikipedia/commons/thumb/6/62/Will-o-the-Wisp_by_Harriet_Hosmer.jpg/170px-Will-o-the-Wisp_by_Harriet_Hosmer.jpg
Error downloading image: http://upload.wikimedia.org/wikipedia/commons/thumb/0/00/Glow_worm_lampyris_noctiluca.jpg/220px-Glow_worm_lampyris_noctiluca.jpg
Error downloading image: http://upload.wikimedia.org/wikipedia/commons/thumb/e/e8/PanellusStipticusAug12_2009.jpg/220px-PanellusStipticusAug12_2009.jpg
Error downloading image: http://upload.wikimedia.org/wikipedia/commons/thumb/9/9e/AmaruMitoPeru.jpg/192px-AmaruMitoPeru.jpg
Error downloading image: http://upload.wikimedia.org/wikipedia/commons/thumb/1/18/Amaru_kero.jpg/220px-Amaru_ker

In this above cell, I ran into problems when trying to read and append the images I got an error like this: "cannot identify image file '/content/images_dragon/Wikipedia.jpg'"

I tried to open the specific image and a viewer, which is not possible, which indicates that the file is broken. 

Therefore I am going to remove any images that may be broken. 
To do this, load in the folder with the images, then I iterate over every single file and try to read it with matplotlibs .imread(). If it works it can be appended to the list, if not it will print out the name of the broken image and remove from the folder. 

In [8]:
import os
import matplotlib.pyplot as plt

# load images in from folder 
data_path = '/content/images_dragons'
files = os.listdir(data_path)

# list to hold dragons, dead and alive
dragons = []
dead_dragons = []

# iterate over the images in the folder 
for file_i in files:
    if '.jpg' in file_i:
        try:     
            image = plt.imread(os.path.join(data_path, file_i))
            dragons.append(image)
        except:
            print(f"Broken image: {file_i}")
            dead_dragons.append(file_i)
            # remove the file
            os.remove(os.path.join(data_path, file_i))

print(f"{len(dead_dragons)} dragons has died.")

Broken image: painted portrait of St.Benedict standing by a desk writing his Rule.jpg
Broken image: Another colour-coded map of north-western Spain.jpg
Broken image: Historical map of northern Spain.jpg
Broken image: Colour-coded linguistic map of north-western Spain.jpg
Broken image: Spray-painting on a road sign.jpg
Broken image: Tees Valley Combined Authority locator map.svg.jpg
Broken image: Stone wall with window of ruined building..jpg
Broken image: line drawing.jpg
Broken image: this is a graph showing the normal distribution of a population as how the diffusion of ideas most likely took place.jpg
Broken image: Present-day linguistic map of Europe.jpg
Broken image: Two illuminations: the pope admonishing a group of people and mounted knights attacking unarmed people with swords.jpg
Broken image: .jpg
Broken image: line drawing of a monster.jpg
Broken image: this is a map showing how and where congregations formed ins the first three centuries.jpg
Broken image: Image of siege of 

In [9]:
# converting remaining dragons to numpy array
dragons = np.array(dragons)

<ipython-input-9-88c0e1615294>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  dragons = np.array(dragons)


In [10]:
# lets look at our data 
for dragon in dragons: 
  print(dragon.shape)
  plt.imshow(dragon)
  plt.show()

Output hidden; open in https://colab.research.google.com to view.

This show that the images are quite varied in both dimensions and size. 

First I am gonna add a colour channel to all the images that don't have one. 

In [11]:
import cv2 

# convert from grayscale to RGB 
for i, dragon in enumerate(dragons):    # iterate over each image 
  if len(dragons[i].shape) == 2:    # check if grayscale
    #dragons[i] = np.expand_dims(dragons[i], axis=2)
    dragons[i] = cv2.cvtColor(dragons[i], cv2.COLOR_GRAY2RGB) # convert to colour aka 3D
  if len(dragons[i].shape) == 3:    # check if image is colour but channels are in different order 
    dragons[i] = cv2.cvtColor(dragons[i],cv2.COLOR_BGR2RGB)
    print(dragon.shape)

(253, 200, 3)
(241, 170, 3)
(147, 220, 3)
(222, 220, 3)
(390, 220, 4)
(165, 220, 3)
(227, 170, 3)
(222, 251, 4)
(330, 220, 3)
(48, 220, 3)
(255, 170, 3)
(310, 220, 3)
(255, 170, 3)
(134, 220, 3)
(145, 330, 3)
(289, 300, 3)
(160, 220, 3)
(310, 220, 3)
(145, 220, 3)
(185, 440, 4)
(106, 220, 3)
(237, 220, 4)
(210, 170, 3)
(221, 300, 3)
(186, 250, 4)
(226, 170, 3)
(165, 220, 3)
(267, 220, 4)
(267, 220, 4)
(255, 170, 3)
(164, 220, 3)
(147, 220, 3)
(267, 220, 3)
(203, 260, 4)
(146, 220, 3)
(267, 220, 4)
(143, 220, 3)
(106, 220, 3)
(188, 250, 3)
(178, 220, 3)
(140, 300, 4)
(383, 280, 4)
(124, 220, 3)
(196, 220, 3)
(165, 220, 3)
(195, 300, 3)


In order to resize the images, I want to look for the smallest and biggest value of both the height and width in order to be able to scale the images up and down to both the largest and biggest image in the array. 

In [12]:
# Initialize variables for smallest and largest height and width
min_height, max_height = dragons[0].shape[0], dragons[0].shape[0]
min_width, max_width = dragons[0].shape[1], dragons[0].shape[1]

# Iterate over the array and get the sizes 
for dragon in dragons:
    height, width = dragon.shape[:2]
    min_height = min(min_height, height)
    max_height = max(max_height, height)
    min_width = min(min_width, width)
    max_width = max(max_width, width)

# Print the results
print(f'Smallest height: {min_height}, largest height: {max_height}')
print(f'Smallest width: {min_width}, largest width: {max_width}')


Smallest height: 48, largest height: 390
Smallest width: 170, largest width: 440


In [14]:
# iterate over each dragon, resize to the smallest dimensions and add back into it's index 
for i, dragon in enumerate(dragons):
  dragons[i] = cv2.resize(dragons[i],(min_height,min_width))       # I can do it like this because the variables keeps updating 
  dragons[i] = dragons[i]

I am gonna use my scraped images as inputs for a stable diffusion variation model. 
This model is a finetuned version of the text-to-image stable diffusion model, so that it can now do image-to-image generation. 
Which is what I am gonna do. 

I am loading the model in via HuggingFace's Transformer Library. 
The official model card can be found here:  https://huggingface.co/lambdalabs/sd-image-variations-diffusers

In [ ]:
pip install diffusers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 737.4/737.4 KB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 26.1 MB/s eta 0:00:00


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 40.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 72.2 MB/s eta 0:00:00


In [ ]:
pip install accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.8/212.8 KB 6.3 MB/s eta 0:00:00


In [ ]:
from diffusers import StableDiffusionImageVariationPipeline
from PIL import Image
import transformers
import torchvision.transforms as transforms
import accelerate

In [ ]:
# choose the device we are on 
device = 'cpu'   
#device = 'cuda:0'

# load in the model 
sd_pipe = StableDiffusionImageVariationPipeline.from_pretrained("lambdalabs/sd-image-variations-diffusers", revision="v2.0")
sd_pipe = sd_pipe.to(device)

# load in input image 
img = Image.open("/content/images_dragons/Mosaico del III secolo a.C. proveniente da Kaulon.jpg")

Fetching 11 files:   0%|          | 0/11 [00:00<?, ?it/s]

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.


In [ ]:
tform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize(
        (224, 224),
        interpolation=transforms.InterpolationMode.BICUBIC,
        antialias=True,
        ),
    transforms.Normalize(
      [0.48145466, 0.4578275, 0.40821073],
      [0.26862954, 0.26130258, 0.27577711]),
])
inp = tform(img).to(device).unsqueeze(0)

out = sd_pipe(inp, guidance_scale=3)
out["images"][0].save("dragon_carving_result.jpg")

  0%|          | 0/50 [00:00<?, ?it/s]